In [ ]:
pip install delta-spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#gerar manifesto
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/AreasVerdes/Gold/gs_gold/`
""")

In [ ]:

spark.sql("""
DROP TABLE IF EXISTS AreasVerdes.gs_gold_presto
""")

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS AreasVerdes.gs_gold_presto (
      Hectares FLOAT,
      CommunityBoard STRING,
      Precinct STRING,
      ZipCode STRING,
      Borough STRING,
      Hectares_por_km2 DOUBLE
 
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/AreasVerdes/Gold/gs_gold/_symlink_format_manifest/'
""")

In [ ]:
spark.stop()